# Error in 3D

In this notebook we load the points OpenPTV creates errors in 3D from 
Robin generated 2D and 3D points, see also proPTV_LineOfSight/generate_error.py

and markers_c0.txt, ...


In [5]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [6]:
class Parameter:
    cams = [0,1,2,3]
    Vmin = [0,0,0]
    Vmax = [300,300,300]
    N1, N2 = 361, 5

In [9]:
params = Parameter()

markers = [np.loadtxt('../proPTV_LineOfSight/markers_c'+str(cam)+'.txt') for cam in params.cams]
XYZ = markers[0][:,2:]
ID = np.argwhere((XYZ[:,0]>-1))[:,0]

In [28]:
ID

array([   0,    1,    2, ..., 1802, 1803, 1804])

In [29]:
xyz = pd.DataFrame(XYZ, columns=['x','y','z'])
xyz['id'] = ID
px.scatter_3d(x=xyz['x'], y=xyz['y'], z=xyz['z'], color=xyz['id']).show()

In [11]:
# First, let's calibrate roughly the cameras

In [33]:

ref_pts = XYZ[[0,721,1409,1462],:]
xyz = pd.DataFrame(ref_pts, columns=['x','y','z'])
xyz['id'] = ID[[0,721,1409,1462]]
px.scatter_3d(x=xyz['x'], y=xyz['y'], z=xyz['z'], color=xyz['id']).show()


In [ ]:
import pathlib, os
working_path = pathlib.Path.cwd()

camera_1_calibration = Calibration().from_file(os.path.join(working_path,"calibration","cam1.tif.ori"), None)
control_parameters = ControlPar().from_file(os.path.join(working_path,"parameters","ptv.par"))
volume_parameters = VolumePar().from_file(os.path.join(working_path,"parameters","criteria.par"))

cal = openptv_python.calibration.Calibration().from_file()
control = openptv_python.parameters.ControlPar().from_file()


In [ ]:

# Fake the image points by back-projection
targets = arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, control.mm),
control,
)

# Jigg the fake detections to give raw_orient some challenge.
targets[:, 1] -= 0.1

self.assertTrue(external_calibration(self.cal, ref_pts, targets, self.control))
np.testing.assert_array_almost_equal(
    self.cal.get_angles(), self.orig_cal.get_angles(), decimal=3
)
np.testing.assert_array_almost_equal(
    self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
)


In [ ]:

    def test_full_calibration(self):
        """Full calibration using clicked points."""
        ref_pts = np.array(
            [
                a.flatten()
                for a in np.meshgrid(np.r_[-60:-30:4j], np.r_[0:15:4j], np.r_[0:15:4j])
            ]
        ).T

        # Fake the image points by back-projection
        targets = arr_metric_to_pixel(
            image_coordinates(ref_pts, self.cal, self.control.get_multimedia_params()),
            self.control,
        )

        # # Full calibration works with TargetArray objects, not NumPy.
        # targets = TargetArray(len(targets))
        # for i, trgt in enumerate(targets):
        #     trgt.set_pnr(i)
        #     trgt.set_pos(targets[i])

        # Perturb the calibration object, then compore result to original.
        self.cal.set_pos(self.cal.get_pos() + np.r_[15.0, -15.0, 15.0])
        self.cal.set_angles(self.cal.get_angles() + np.r_[-0.5, 0.5, -0.5])


        self.orient_par.ccflag=0
        self.orient_par.xhflag=0
        self.orient_par.yhflag=0
        print(f"Calibrating with the following flags: {self.orient_par}")

        _, _, _ = full_calibration(
            self.cal,
            ref_pts,
            targets,
            self.control,
            self.orient_par
            )

        np.testing.assert_array_almost_equal(
            self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
        )
        np.testing.assert_array_almost_equal(
            self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
        )

        print(f"{self.cal.get_pos()}")
        print(f"{self.cal.get_angles()}")
        print(f"{self.cal.added_par}")

        # Perturb the calibration object, then compore result to original.
        self.cal.set_pos(self.cal.get_pos() + np.r_[1.0, -1.0, 1.0])
        self.cal.set_angles(self.cal.get_angles() + np.r_[-0.1, 0.1, -0.1])

        self.orient_par.ccflag=1
        self.orient_par.xhflag=1
        self.orient_par.yhflag=1
        print(f"Calibrating with the following flags: {self.orient_par}")

        _, _, _ = full_calibration(
            self.cal,
            ref_pts,
            targets,
            self.control,
            self.orient_par
            )

        np.testing.assert_array_almost_equal(
            self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
        )
        np.testing.assert_array_almost_equal(
            self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
        )

        print(f"{self.cal.get_pos()}")
        print(f"{self.cal.get_angles()}")
        print(f"{self.cal.added_par}")

        # Perturb the calibration object, then compore result to original.
        # self.cal.set_pos(self.cal.get_pos() + np.r_[1.0, -1.0, 1.0])
        # self.cal.set_angles(self.cal.get_angles() + np.r_[-0.1, 0.1, -0.1])

        self.orient_par.ccflag=0
        self.orient_par.xhflag=0
        self.orient_par.yhflag=0
        self.orient_par.k1flag=0
        self.orient_par.k2flag=0
        self.orient_par.k3flag=0
        self.orient_par.scxflag=0
        self.orient_par.sheflag=0
        print(f"Calibrating with the following flags: {self.orient_par}")

        _, _, _ = full_calibration(
            self.cal,
            ref_pts,
            targets,
            self.control,
            self.orient_par
            )

        np.testing.assert_array_almost_equal(
            self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
        )
        np.testing.assert_array_almost_equal(
            self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
        )
        print(f"{self.cal.get_pos()}")
        print(f"{self.cal.get_angles()}")
        print(f"{self.cal.added_par}")

        self.orient_par.ccflag=0
        self.orient_par.xhflag=0
        self.orient_par.yhflag=0
        self.orient_par.k1flag=0
        self.orient_par.k2flag=0
        self.orient_par.k3flag=0
        self.orient_par.scxflag=1
        self.orient_par.sheflag=0
        print(f"Calibrating with the following flags: {self.orient_par}")

        _, _, _ = full_calibration(
            self.cal,
            ref_pts,
            targets,
            self.control,
            self.orient_par
            )

        np.testing.assert_array_almost_equal(
            self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
        )
        np.testing.assert_array_almost_equal(
            self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
        )
        print(f"{self.cal.get_pos()}")
        print(f"{self.cal.get_angles()}")
        print(f"{self.cal.added_par}")

        self.orient_par.ccflag=0
        self.orient_par.xhflag=0
        self.orient_par.yhflag=0
        self.orient_par.k1flag=0
        self.orient_par.k2flag=0
        self.orient_par.k3flag=1
        self.orient_par.scxflag=0
        self.orient_par.sheflag=0
        print(f"Calibrating with the following flags: {self.orient_par}")

        _, _, _ = full_calibration(
            self.cal,
            ref_pts,
            targets,
            self.control,
            self.orient_par
            )

        np.testing.assert_array_almost_equal(
            self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
        )
        np.testing.assert_array_almost_equal(
            self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
        )
        print(f"{self.cal.get_pos()}")
        print(f"{self.cal.get_angles()}")
        print(f"{self.cal.added_par}")

        self.orient_par.ccflag=0
        self.orient_par.xhflag=0
        self.orient_par.yhflag=0
        self.orient_par.k1flag=0
        self.orient_par.k2flag=1
        self.orient_par.k3flag=0
        self.orient_par.scxflag=0
        self.orient_par.sheflag=0
        print(f"Calibrating with the following flags: {self.orient_par}")

        _, _, _ = full_calibration(
            self.cal,
            ref_pts,
            targets,
            self.control,
            self.orient_par
            )

        np.testing.assert_array_almost_equal(
            self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
        )
        np.testing.assert_array_almost_equal(
            self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
        )
        print(f"{self.cal.get_pos()}")
        print(f"{self.cal.get_angles()}")
        print(f"{self.cal.added_par}")

        self.orient_par.ccflag=0
        self.orient_par.xhflag=0
        self.orient_par.yhflag=0
        self.orient_par.k1flag=1
        self.orient_par.k2flag=0
        self.orient_par.k3flag=0
        self.orient_par.scxflag=0
        self.orient_par.sheflag=0
        print(f"Calibrating with the following flags: {self.orient_par}")

        _, _, _ = full_calibration(
            self.cal,
            ref_pts,
            targets,
            self.control,
            self.orient_par
            )

        np.testing.assert_array_almost_equal(
            self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
        )
        np.testing.assert_array_almost_equal(
            self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
        )
        print(f"{self.cal.get_pos()}")
        print(f"{self.cal.get_angles()}")
        print(f"{self.cal.added_par}")

        self.orient_par.ccflag=0
        self.orient_par.xhflag=0
        self.orient_par.yhflag=0
        self.orient_par.k1flag=0
        self.orient_par.k2flag=0
        self.orient_par.k3flag=1
        self.orient_par.scxflag=0
        self.orient_par.sheflag=0
        self.orient_par.p1flag=1
        self.orient_par.p2flag=0

        print(f"Calibrating with the following flags: {self.orient_par}")

        _, _, _ = full_calibration(
            self.cal,
            ref_pts,
            targets,
            self.control,
            self.orient_par
            )

        np.testing.assert_array_almost_equal(
            self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
        )
        np.testing.assert_array_almost_equal(
            self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
        )
        print(f"{self.cal.get_pos()}")
        print(f"{self.cal.get_angles()}")
        print(f"{self.cal.added_par}")

        self.orient_par.ccflag=1
        self.orient_par.xhflag=0
        self.orient_par.yhflag=0
        self.orient_par.k1flag=0
        self.orient_par.k2flag=0
        self.orient_par.k3flag=0
        self.orient_par.scxflag=0
        self.orient_par.sheflag=0
        self.orient_par.p1flag=1
        self.orient_par.p2flag=1
        print(f"Calibrating with the following flags: {self.orient_par}")

        _, _, _ = full_calibration(
            self.cal,
            ref_pts,
            targets,
            self.control,
            self.orient_par
            )

        np.testing.assert_array_almost_equal(
            self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
        )
        np.testing.assert_array_almost_equal(
            self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
        )
        print(f"{self.cal.get_pos()}")
        print(f"{self.cal.get_angles()}")
        print(f"{self.cal.added_par}")

In [ ]:
def point_position(
    targets: np.ndarray,
    num_cams: int,
    multimed_pars: MultimediaPar,
    cals: List[Calibration],
) -> Tuple[float, np.ndarray]:
    """
    Calculate an average 3D position implied by the rays.

    sent toward it from cameras through the image projections of the point.

    Arguments:
    ---------
    targets - for each camera, the 2D metric, flat, centred coordinates
        of the identified point projection.
    multimed_pars - multimedia parameters struct for ray tracing through
        several layers.
    cals - each camera's calibration object.

    Returns
    -------
    A tuple containing the ray convergence measure (an average of skew ray distance across all ray pairs)
    and the average 3D position vector.
    """
    # loop counters
    num_used_pairs = 0
    dtot = 0.0
    point_tot = np.array([0.0, 0.0, 0.0])

    vertices = np.zeros((num_cams, 3))
    directs = np.zeros((num_cams, 3))
    point = np.zeros(3)

    # Shoot rays from all cameras.
    for cam in range(num_cams):
        if targets[cam][0] != COORD_UNUSED:
            vertices[cam], directs[cam] = ray_tracing(
                targets[cam][0], targets[cam][1], cals[cam], multimed_pars
            )

    # Check intersection distance for each pair of rays and find position
    for cam in range(num_cams):
        if targets[cam][0] == COORD_UNUSED:
            continue

        for pair in range(cam + 1, num_cams):
            if targets[pair][0] == COORD_UNUSED:
                continue

            num_used_pairs += 1
            tmp, point = skew_midpoint(
                vertices[cam], directs[cam], vertices[pair], directs[pair]
            )
            dtot += tmp
            point_tot += point

    res = point_tot / num_used_pairs
    dtot /= num_used_pairs

    return float(dtot), res.astype(float)

In [8]:
for i in range(1,6):
    df = pd.read_csv(f'./reconstruction/plane{i}.123456789',skiprows=1,sep='\s+',header=None)
    df.columns = ['id','x','y','z','i0','i1','i2','i3']
    # df.head()
    ground = pd.read_csv(f'./calibration/plane_{i}.csv',header=None)
    ground.columns = ['id','x','y','z']
    # ground.head()


    sc = px.scatter_3d(df, x='x', y='y', z='z',color='id')
    fig2 = go.Figure(data=[go.Scatter3d(x=ground.x, y=ground.y, z=ground.z,
                                    mode='markers')])
    fig2.add_traces(sc.data)
    fig2.show()
